In [ ]:
from flask import Flask # 플라스크 클래스 임포트
from flask import request, redirect
import pickle # 데이터 저장/로딩을 위한 모듈
import pandas as pd
import numpy as np

app = Flask(__name__) # 내장변수 name을 이용해 서버를 구동시키는 객체 생성

# 코사인 유사도 측정 결과 불러오기
with open('collaborative_filtering.pkl','rb') as f :
    df = pickle.load(f)

# 사용자 ID(행)와 책 이름(열)으로 이루어진 데이터프레임 불러오기
with open('book_data.pkl','rb') as f :
    data = pickle.load(f)

# 책 목록(리스트) 불러오기
with open('book_list.pkl','rb') as f :
    book_list = pickle.load(f)

    
@app.route("/mahang/bookrecommend", methods=["GET",'POST'])
def book_recommend() : 
    if request.method == 'POST' : # POST 방식일 경우
        recom_list = preprocessing(request.form, df, data, book_list)
        return redirect("http://localhost:8081/project2/bookrecommended?book1="
                                +recom_list[0]+"&book2="+recom_list[1]+"&book3="+recom_list[2])
    else : # GET 방식일 경우
        recom_list = preprocessing(request.args, df, data, book_list)
        return redirect("http://localhost:8081/project2/bookrecommended?book1="
                                +recom_list[0]+"&book2="+recom_list[1]+"&book3="+recom_list[2])

if __name__ == "__main__" : # .py 파일에서 main함수 역할
    app.run(host="localhost", port="9000") # 9000번 포트로 서버 구동

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Sep/2020 09:14:15] "GET /mahang/bookrecommend?id=whizzerscowl&bookname=심판 HTTP/1.1" 302 -
127.0.0.1 - - [25/Sep/2020 09:14:23] "GET /mahang/bookrecommend?id=whizzerscowl&bookname=기억+1 HTTP/1.1" 302 -
127.0.0.1 - - [25/Sep/2020 09:16:37] "GET /mahang/bookrecommend?id=whizzerscowl&bookname=돈의+속성 HTTP/1.1" 302 -
127.0.0.1 - - [25/Sep/2020 09:17:06] "GET /mahang/bookrecommend?id=whizzerscowl&bookname=규칙+없음 HTTP/1.1" 302 -
127.0.0.1 - - [25/Sep/2020 09:18:52] "GET /mahang/bookrecommend?id=whizzerscowl&bookname=규칙+없음 HTTP/1.1" 302 -
127.0.0.1 - - [25/Sep/2020 09:19:31] "GET /mahang/bookrecommend?id=whizzerscowl&bookname=규칙+없음 HTTP/1.1" 302 -
127.0.0.1 - - [25/Sep/2020 09:19:41] "GET /mahang/bookrecommend?id=whizzerscowl&bookname=심판 HTTP/1.1" 302 -
127.0.0.1 - - [25/Sep/2020 09:19:49] "GET /mahang/bookrecommend?id=whizzerscowl&bookname=심판 HTTP/1.1" 302 -
127.0.0.1 - - [25/Sep/2020 09:21:15] "GET /mahang/bookrecommen

127.0.0.1 - - [25/Sep/2020 15:32:27] "GET /mahang/bookrecommend?id=whizzerscowl&bookname=하버드+상위+1퍼센트의+비밀+%28리커버+에디션%29%09 HTTP/1.1" 500 -
127.0.0.1 - - [25/Sep/2020 15:32:31] "GET /mahang/bookrecommend?id=whizzerscowl&bookname=하버드+상위+1퍼센트의+비밀+%28리커버+에디션%29 HTTP/1.1" 302 -
[2020-09-25 15:35:05,255] ERROR in app: Exception on /mahang/bookrecommend [GET]
Traceback (most recent call last):
  File "C:\Users\SMT031\anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 2646, in get_loc
    return self._engine.get_loc(key)
  File "pandas\_libs\index.pyx", line 111, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 133, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 163, in pandas._libs.index.IndexEngine._get_loc_duplicates
  File "pandas\_libs\index.pyx", line 180, in pandas._libs.index.IndexEngine._maybe_get_bool_indexer
KeyError: '기'

During handling of the above exception, another exception occurred:

Traceback (most re

In [2]:
# 사용자의 안 읽은 책 목록 가져오기
def get_unread_books(data, userID, book_list):
    user_rating = data.loc[userID,:]
    already_read = user_rating[user_rating > 0].index.tolist()
    unread_list = [book for book in book_list if book not in already_read]
    return unread_list

In [3]:
# 사용자가 읽고 평가한 책이름을 기준으로 안 읽은 책 중 도서 5권 추천
def get_recommendation(df, book_name, data, userID, book_list):
    return df[book_name][get_unread_books(data, userID, book_list)].sort_values(ascending=False)[1:6].index.tolist()

In [4]:
# 웹에서 입력받은 ID와 값으로 도서를 추천할 최상위함수
def preprocessing(data_dict, df, data, book_list):
    book_name = data_dict['bookname']
    userID = data_dict['id']
    
    return get_recommendation(df, book_name, data, userID, book_list)